In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

G=nx.MultiDiGraph()
G.add_nodes_from(range(0,9))
nodes_position = {0:(0,2),1:(0,1),2:(1,1),3:(2,1),4:(2,0),5:(1,0),6:(0,0),7:(1,2),8:(2,2)}
G.add_edges_from([(1,0),(1,2),(2,3),(3,4)],weight=1)
G.add_edges_from([(4,5),(5,6),(6,1)],weight=1)
G.add_edges_from([(0,7),(7,8),(8,3)],weight=1)

G.add_edges_from([(5,2),(2,5),(7,2),(2,7)], weight=2)

# Add variability to the edges

edges = G.edges()
for u,v in edges:
    edge_data = G.get_edge_data(u, v,0)
    val = G[u][v][0]['weight']
    G[u][v][0]['weight'] +=  np.random.randn(1)[0]*val/10
    
H = nx.line_graph(G,create_using=nx.MultiDiGraph)

H.add_nodes_from((node, G.edges[node]) for node in H)

edges = G.edges()  
edges = H.edges

# Add weights to the links of the dual graph. Let's define it as the average weight 
# of the two edges it connects in the primal.
for ed in edges:
    H.edges[ed]['weight'] = 0.5*(H.nodes[ed[0]]['weight']+H.nodes[ed[1]]['weight'])

In [ ]:
# Relabel nodes to avoid cumbersome notation
mapping = {}
c = 0
for node in H:
    mapping[node]=c
    c+=1

H2 = nx.relabel_nodes(H,mapping)

In [ ]:
# Redefine the position as the middle point between the nodes it
# connects in the primal. For plotting purposes only
nodes_position_dual = {}  
for node in H2:
    u, v,_ = list(H.nodes())[node]   
    nodes_position_dual[node]=(0.5*(nodes_position[u][0]+nodes_position[v][0]),
                               0.5*(nodes_position[u][1]+nodes_position[v][1]))    

In [ ]:
# Redefine the nodes position of the bidirectional edges because we will bend them:
nodes_position_dual[6]=(0.92,0.5)
nodes_position_dual[10]=(1.08,0.5)

nodes_position_dual[7]=(1.08,1.5)
nodes_position_dual[2]=(0.92,1.5)

In [ ]:
# Extract the edge and the weight values as arrays from primal
widths = nx.get_edge_attributes(G, 'weight')
edge_list = list(widths.keys())
weight_list = list(widths.values())

In [ ]:
# Extract the edge and the weight values as arrays from dual
widths_2 = nx.get_edge_attributes(H2, 'weight')
edge_list_2 = list(widths_2.keys())
weight_list_2 = list(widths_2.values())

In [ ]:
# Concatenate lists of unidirectional edges
unidir = edge_list[0:4]+edge_list[6:9]+edge_list[10:12]+edge_list[13:]
unidir_weight = weight_list[0:4]+weight_list[6:9]+weight_list[10:12]+weight_list[13:]

In [ ]:
# Concatenate lists of bidirectional edges
bidir = edge_list[4:6]+edge_list[9:10]+edge_list[12:13]
bidir_weight = weight_list[4:6]+weight_list[9:10]+weight_list[12:13]

In [ ]:
# Plot the whole graph
nx.draw_networkx_nodes(G,pos=nodes_position,
                       node_size=20,
                      node_color='black')
# Unidirectional edges
nx.draw_networkx_edges(G,pos=nodes_position,
                       edgelist = unidir,
                       node_size=20,
                       width=unidir_weight,
                       edge_color='black')

# bidirectional edges
nx.draw_networkx_edges(G,pos=nodes_position,
                       node_size=20,
                       edgelist = bidir,
                       width=bidir_weight,
                       edge_color='black',
                       connectionstyle="arc3,rad=0.2")

nx.draw_networkx_nodes(H2,pos=nodes_position_dual,
                       node_color='red',
                       node_size=20,
                      alpha= 0.3)
nx.draw_networkx_edges(H2,pos=nodes_position_dual,                       
                       edge_color='red',
                      connectionstyle="arc3,rad=0.4",
                      alpha = 0.3,
                        edgelist = edge_list_2,
                       width=weight_list_2)


# Calculate de adjacency matrix if the dual graph
W = nx.adjacency_matrix(H,weight='weight').todense()
numNodes = H.number_of_nodes()

# Define Laplacian Matrix
def Laplacian_Matrix(W):
    return np.diag(np.sum(W, axis = 1)) - W.T

Lout = Laplacian_Matrix(W)

In [ ]:
# First protocol, Fig 3

xMax = 2
h = 1e-3

bVector = [0.4,0.7,1]
tSimul = np.arange(0,20,h)
alpha = 0.5*np.ones(numNodes)
AlphaMatrix = np.diag(alpha)


plt.figure(dpi=300)

j = 1

Aalpha = -(Lout+AlphaMatrix)

for bVal in bVector:

    bu = bVal*np.ones(numNodes)
    xStar = -np.linalg.inv(Aalpha)@bu

    x = np.zeros((len(tSimul),numNodes))

    for i in range(0,len(tSimul)-1):

        nodes_occupied = np.argwhere(x[i,:]>xMax)
        Woccupied = np.copy(W)
        Woccupied[:,nodes_occupied] = 0 
        LoutOccupied = Laplacian_Matrix(Woccupied)
        x[i+1,:] = x[i,:] + h*(-(LoutOccupied+AlphaMatrix)@x[i,:]+bu)

    plt.subplot(1,3,j)    

    plt.title('b = ' + str(np.round(bVal,2)))
    lines= plt.plot(tSimul,x,'-',alpha=0.7)          
    #if(j==1):
        #for k in range(len(lines)):
            #linecolor = lines[k].get_color()  
            #plt.plot(tSimul,xStar[k]*np.ones(len(tSimul)),'--',c=linecolor)  
    plt.plot(tSimul,xMax*np.ones(len(tSimul)),'k-')
    plt.xlabel('time')
    plt.ylabel(r'$u_i(t)$')
    j+=1

plt.tight_layout()
resolution_value = 300


In [ ]:
# Numerical Order Parameter as a function of b fixing xMax = 2

xMax = 2.
h = 5e-3
tSimul = np.arange(0,24,h)
inputConstant = np.linspace(0.2,1,50)
orderParameter1 = np.zeros(len(inputConstant))
meanDensity = np.zeros(len(inputConstant))
meanFlow = np.zeros(len(inputConstant))

alpha = 0.5*np.ones(numNodes)
AlphaMatrix = np.diag(alpha) 
Lout = Laplacian_Matrix(W)
Aalpha = -(Lout+AlphaMatrix)

# Loop over different values of b
for k in range(len(inputConstant)):

    bu = inputConstant[k]*np.ones(numNodes)    
    xStar = -np.linalg.inv(Aalpha)@bu    
    xJam = np.linalg.inv(AlphaMatrix)@bu        

    # These variables accumulate the values of x and flow for
    # calculation of fundamental diagram
    x = np.zeros((len(tSimul),numNodes))    
    x[0,:] = np.zeros(numNodes)
    flow = np.zeros((len(tSimul),numNodes))
    flow[0,:] = np.zeros(numNodes)    


    for i in range(0,len(tSimul)-1):
        # Identify occupied nodes
        nodes_occupied = np.argwhere(x[i,:]>xMax) 
        # Redefine connectivity matrix and Laplacian in occupied states
        Woccupied = np.copy(W)
        Woccupied[:,nodes_occupied] = 0 
        LoutOccupied = Laplacian_Matrix(Woccupied)
        x[i+1,:] = x[i,:] + h*(-(LoutOccupied+AlphaMatrix)@x[i,:]+bu)
        flow[i+1,:] = Woccupied@x[i+1,:]
        
    # Average value of x over the last 1000 steps (5 units of time)    
    xFinal = np.mean(x[-1000:,:],axis=0)    
    # Order parameter as defined in Eq. 28
    orderParameter1[k]=np.linalg.norm(xFinal-xStar)/np.linalg.norm(xStar-xJam)
    meanDensity[k] = np.mean(xFinal)
    meanFlow[k]=np.mean(flow[-1000:,:])
    

In [ ]:
# This is for the analytical calculation of the transitions:
from scipy.optimize import fsolve

xMax = 2.

# Equation for the first phase transition (free-flow to congestion)
def equation1(x,xmax,A,numNodes):
    b = x*np.ones(numNodes)
    return np.max(-np.linalg.inv(A)@b)-xmax

# Equation for the second phase transition (congestion to total jam)
def equation2(x,xmax,A,numNodes):
    b = x*np.ones(numNodes)
    return np.min(-np.linalg.inv(A)@b)-xmax

alpha = 0.5*np.ones(numNodes)
AlphaMatrix = np.diag(alpha)

Lout = Laplacian_Matrix(W)

# The Amatrices for the two pahse transitions
A1 = -(Lout+AlphaMatrix)
A2 = -AlphaMatrix

# The two solutions
bSol = fsolve(equation1,0.3,args=(xMax,A1,numNodes))
bSol2 = fsolve(equation1,0.5,args=(xMax,A2,numNodes))

In [ ]:
# Auxiliary functions to calculate the whole order parameter in a self-consistent way
def selfConsistentXStar(x,idCong,A0,b,AlphaMatrix,xMax): 
    if(len(idCong)==0):        
        Atot = Amatrix(A0)        
        return (Atot - AlphaMatrix)@x + b
    else:       
        xMult = np.copy(x)
        Atot = Amatrix(A0)
        A1 = np.copy(A0)
        for i in range(len(idCong)):
            idx = idCong[i]

            A1[:,idx] = 0
            xMult[idx] = xMax
            
            d = x[idx]
            Atot = d*Atot + (1-d)*Amatrix(A1)

        return (Atot-AlphaMatrix)@xMult+b

def selfConsistentCase3(x,idCong,A0,b,AlphaMatrix,xMax):     
    xMult = np.copy(x)
    idx1 = idCong[0]
    idx2 = idCong[1]
    
    A1 = np.copy(A0)
    A1[:,idx1] = 0
    xMult[idx1] = xMax
    
    A2 = np.copy(A1)
    A2[:,idx2] = 0
    xMult[idx2] = xMax
    
    d1 = x[idx1]
    d2 = x[idx2]
    
    aux = d1*Amatrix(A0)+(1-d1)*Amatrix(A1)
    aux2 = d2*aux+(1-d2)*Amatrix(A2)
    
    f = (aux2-AlphaMatrix)@xMult
    f += b    
    
    return f    

def Amatrix(A):    
    return -Laplacian_Matrix(A)
    

In [ ]:
# This is the analytical self-consistent solution for the order parameter as a funciton of b.

bVec = np.linspace(0.2,1.0,100)
rhoVec = np.zeros_like(bVec)
DensityVec = np.zeros_like(bVec)
VelocityVec = np.zeros_like(bVec)
idCong = []
alpha = 0.5*np.ones(numNodes)
AlphaMatrix = np.diag(alpha)
W0 = np.copy(W)    
Anaive = Amatrix(W0)

bcritical1 = fsolve(equation1,0.3,args=(xMax,Anaive-AlphaMatrix,numNodes))
bcritical2 = fsolve(equation2,0.5,args=(xMax,-AlphaMatrix,numNodes))

for kk in range(len(bVec)):
    
    b = bVec[kk]
    bu = b*np.ones(numNodes)
    
    if(b<bcritical1):
        rhoVec[kk] = 0   
        xFinal = -np.linalg.inv(Anaive-AlphaMatrix)@bu
        DensityVec[kk] = np.mean(xFinal)
        VelocityVec[kk] = np.mean(W0@xFinal)
    elif(b>bcritical2):
        rhoVec[kk] = 1
        xFinal = np.linalg.inv(AlphaMatrix)@bu  
        DensityVec[kk] = np.mean(xFinal)
        VelocityVec[kk] = 0
    else:
        bu = b*np.ones(numNodes)

        Anaive = Amatrix(W0)
        xStar = -np.linalg.inv(Anaive-AlphaMatrix)@bu
        xJam = np.linalg.inv(AlphaMatrix)@bu  

        xStarSorted = np.sort(xStar)
        idCongTot = []
        for q in range(len(xStarSorted)):
            if(xStarSorted[q]>xMax):
                idx = np.argwhere(xStar==xStarSorted[q])
                idCongTot.append(idx[0,0])                
        idCongTot = idCongTot[::-1]

        if(len(idCongTot)==0):
            x0 = np.ones(numNodes)
            xFinal = fsolve(selfConsistentXStar,x0,args=([],W0,bu,AlphaMatrix,xMax))

        else:    
            idCong = []
            while(len(idCongTot)>0): 
                idCong.append(idCongTot[0])
                x0 = np.ones(numNodes)
                x = fsolve(selfConsistentXStar,x0,args=(idCong,W0,bu,AlphaMatrix,xMax))        

                xFinal = np.copy(x)
                for i in range(len(idCong)):
                    idx = idCong[i]
                    xFinal[idx] = xMax

                xStarSorted = np.sort(xFinal)
                idCongTot = []        
                for i in range(len(xStarSorted)):
                    if(xStarSorted[i]>xMax):
                        idx = np.argwhere(xFinal==xStarSorted[i])                      
                        idCongTot.append(idx[0,0])                
                idCongTot = idCongTot[::-1]                        

        
        Atot = np.copy(W0)
        Atot[:,idCong] = 0

        rhoVec[kk] = np.linalg.norm(xFinal-xStar)/np.linalg.norm(xStar-xJam) 
        DensityVec[kk] = np.mean(xFinal)
        VelocityVec[kk] = np.mean(Atot@xFinal)

In [ ]:
# Solution of the two critical transitions for several uMax and a_i

from scipy.optimize import fsolve

def equation1(x,xmax,A,numNodes):
    b = x*np.ones(numNodes)
    return np.max(-np.linalg.inv(A)@b)-xmax

alphaVec = np.linspace(0.1,5,100)

xMaxVec = [1,2,3]
bSolution1 = np.zeros((len(alphaVec),len(xMaxVec)))
bSolution2 = np.zeros((len(alphaVec),len(xMaxVec)))

for k in range(len(xMaxVec)):
    xMax = xMaxVec[k]

    for i in range(len(alphaVec)):
        alphaVal = alphaVec[i]
        alpha = alphaVal*np.ones(numNodes)
        AlphaMatrix = np.diag(alpha)    
        Lout = Laplacian_Matrix(W)
        AA = -(Lout+AlphaMatrix)
        AA2 = -AlphaMatrix

        bSolution1[i,k] = fsolve(equation1,0.3,args=(xMax,AA,numNodes))
        bSolution2[i,k] = fsolve(equation2,0.3,args=(xMax,AA2,numNodes))
        


In [ ]:
# Generates Figures 4a, 4b and 4c
fig = plt.figure(dpi=300)

gs = fig.add_gridspec(2,2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[:, 1])

ax1.plot(inputConstant,orderParameter1,'o--')
ax1.plot(bVec,rhoVec,'-r')
ax1.plot([bSol,bSol],[0,1],'-k')
ax1.plot([bSol2,bSol2],[0,1],'--k')
ax1.set_xlabel('$b$')
ax1.set_ylabel(r'$\rho$')
ax1.set_xlim(0.2,1.01)
ax1.set_ylim(-0.01,1.01)
ax1.legend(['Simul.','Theory'])


ax2.plot(meanDensity,meanFlow,'o--')
ax2.set_xlabel('Density')
ax2.set_ylabel('Flow')
ax2.set_xlim(meanDensity[0],meanDensity[-1])


ax3.plot(alphaVec,bSolution1,'-')
ax3.set_prop_cycle(None)
ax3.plot(alphaVec,bSolution2,'--')
ax3.set_xlabel(r'$\alpha$')
ax3.set_ylabel(r'$b$')
ax3.set_xlim(alphaVec[0],alphaVec[-1])
ax3.legend([r'$x_{max}=1$',r'$x_{max}=2$',r'$x_{max}=3$'])
plt.tight_layout()



In [ ]:
# Optimization
from scipy.optimize import minimize
from scipy.sparse import csr_array,csc_array,diags_array
from scipy.sparse.linalg import inv

# This is the optimization problem, Eq. 32a written in a sligthly different way as we
# make use of the fact that xMax = Constant for all nodes, hence the optimization becomes:
def minim_func(dw,indices,A0,b,numNodes):            
    DeltaW = csr_array((dw,indices),shape=(numNodes,numNodes)).todense()    
    AW = -Laplacian_Matrix(DeltaW)    
    Ahat = AW+A0
    xEq = -np.linalg.inv(Ahat)@b
        
    return np.max(np.abs(xEq))

# The simple expression allows to calculate the Gradient of minimizing Function
def grad_func(dw,indices,A0,b,numNodes):
    grad_vector = np.zeros(len(dw))    
    DeltaW = csr_array((dw,indices),shape=(numNodes,numNodes)).todense()
    AW = -Laplacian_Matrix(DeltaW)    
    Ahat = AW+A0
    AhatInv = np.linalg.inv(Ahat)   
    xEq = -AhatInv@b
        
    iMax = np.argmax(np.abs(xEq))    
    
    dJdx = np.zeros(len(xEq))
    dJdx[iMax]=np.sign(xEq[iMax])
    
    for q in range(len(dw)):
        i = indices[0][q]
        j = indices[1][q]        
        dAHat_dwij = np.zeros((numNodes,numNodes))
        dAHat_dwij[j,i] = 1
        dAHat_dwij[i,i] = -1
        aux = AhatInv@dAHat_dwij@AhatInv
        
        grad_vector[q] = dJdx@(aux@b)        
        
    return grad_vector

In [ ]:
# Select the edges of the intervention Graph -we assume all of them can be intervened-
indices = W.nonzero()
# Matrices as usual
Lout = Laplacian_Matrix(W)
alpha = 0.5*np.ones(numNodes)
# Use a value of b guaranteeing sub-threshold regime
bVal = 0.4

AlphaMatrix = np.diag(alpha)
b = bVal*np.ones(numNodes)

A0 = -(Lout+AlphaMatrix)

# numer of links in the intervention graph
n = len(indices[0])

# Number of realizations for the optimization
numIter = 10

# Vector to check solution at different levels of resoruces available
wTot_Vec = np.linspace(0.1,40,50)

# Initilize arrays where data will be stored
bestXGrad = np.zeros((len(wTot_Vec),numIter))
bestSolution = np.zeros((len(wTot_Vec),n))
lastBest = 1e6*np.ones(len(wTot_Vec))

for outLoop in range(numIter):
    print(outLoop)
    for k in range(len(wTot_Vec)):
        # Random Initialization of the links in intervention graph
        dw0 = np.random.rand(n,)

        # available resources 
        w_tot = wTot_Vec[k]        

        # Restrictions over resoruces
        cons = ({'type': 'eq', 'fun': lambda x:  w_tot-np.sum(x)})
        # Only positive interventions allowed
        bounds = [(0, None) for i in range(n)]
        
        # Guaranteeing that initial guess sums to wTot
        dw0 /= np.sum(dw0)
        dw0 *= w_tot
        
        # Apply SLSQP optimizer 
        resultGrad = minimize(minim_func, x0=dw0, constraints=cons,jac = grad_func, bounds=bounds,
            options={"maxiter" : 500}, method='SLSQP', args=(indices,A0,b,numNodes))
        
        # Extract optimization result
        dw0Grad = resultGrad.x
        
        # Reconstruct the Intervention Graph 
        DeltaW = csr_array((dw0Grad,indices),shape=(numNodes,numNodes)).todense()  
        AW = -Laplacian_Matrix(DeltaW)
        # A matrix after intervention
        Ahat = AW+A0
        # Equilibrium point after intervention
        x1Grad = -np.linalg.inv(Ahat)@b
        
        # maximum equilibrium is the minimizing functional when xmax is constant for all nodes, I extract it
        bestXGrad[k,outLoop] = np.max(x1Grad) 
        
        # Store best result across the itrations
        if(np.max(x1Grad)<lastBest[k]):
            bestSolution[k,:]=dw0Grad
            lastBest[k]=np.max(x1Grad)


In [ ]:
# This is the non-negative least-squares solution that we can find assuming equation 33.
from scipy.optimize import nnls

edgeOrder = list(zip(indices[0],indices[1]))
Lout = Laplacian_Matrix(W)

B = nx.incidence_matrix(H2,oriented=True,edgelist=edgeOrder).todense()
AlphaMatrix = np.diag(alpha)
b = bVal*np.ones(numNodes)
A0 = -(Lout+AlphaMatrix)
xStar = -np.linalg.inv(A0)@b  
xMin = np.sum(xStar)/numNodes 

bb = -b/xMin - np.sum(A0,axis=1)
solut = nnls(B,bb)

nnls_total_intervention = np.sum(solut[0])

In [ ]:
# We check out of the numerical optimization performed previously, which value of available resources
# correspond to the nnls solution

idBest = np.argmin(np.abs(wTot_Vec-nnls_total_intervention))

# Extract the solution of the optimization that coincides with the optimal one
dw0Best = bestSolution[idBest+1,:]
        
# Reconstruct the intervention graph 
DeltaW = csr_array((dw0Best,indices),shape=(numNodes,numNodes)).todense()  

# And calculate the relative change of the original graph and the intervention one
maskedW = np.zeros_like(DeltaW)
for i in range(len(DeltaW)):
    for j in range(len(DeltaW)):
        if(W[i,j]>0):
            maskedW[i,j] = DeltaW[i,j]/W[i,j]

H3 = nx.from_numpy_array(maskedW,create_using=nx.DiGraph)
widths3 = nx.get_edge_attributes(H3, 'weight')

In [ ]:
# How the critical transition point form free-flow to congestion changes at varying available intervention
# resources:

traffic_onset = np.zeros_like(wTot_Vec)
total_congestion = np.zeros_like(wTot_Vec)

for i in range(len(wTot_Vec)):
    # Extract the optimization solution
    dw0 = bestSolution[i,:]
        
    # Intervention Graph Reconstruction 
    DeltaW = csr_array((dw0,indices),shape=(numNodes,numNodes)).todense()  
    
    # Resulting Adjacency matrix     
    W_adj = W+DeltaW
    
    AlphaMatrix = np.diag(alpha)    
    Lout = Laplacian_Matrix(W_adj)
    AA = -(Lout+AlphaMatrix)
    AA2 = -AlphaMatrix

    # Calculate the two transition points.
    traffic_onset[i] = fsolve(equation1,0.3,args=(xMax,AA,numNodes))
    total_congestion[i]= fsolve(equation2,0.3,args=(xMax,AA2,numNodes))
    

In [ ]:
# Generates Fig. 4d and 4e
fig = plt.figure(dpi=300)

gs = fig.add_gridspec(2,2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[:, 1])


ax1.plot(wTot_Vec,np.min(bestXGrad,axis=1)-2,marker='o', markersize=4)
ax1.plot(wTot_Vec,0.8*np.ones_like(wTot_Vec)-2,'--r')
ax1.plot([nnls_total_intervention,nnls_total_intervention],[-1.3,-0.4],'--k')
ax1.set_xlim(wTot_Vec[0],wTot_Vec[-1])
ax1.set_ylim(-1.3,-0.4)
ax1.set_xlabel(r'$w_{tot}$')
ax1.set_ylabel(r'$J^*$')


ax2.plot(wTot_Vec,traffic_onset,marker='o', markersize=4)
ax2.plot([nnls_total_intervention,nnls_total_intervention],[0.8,1.55],'--k')
ax2.set_xlabel(r'$w_{tot}$')
ax2.set_ylabel(r'$b_c$')
ax2.set_xlim(wTot_Vec[0],wTot_Vec[-1])
ax2.set_ylim(0.8,1.55)

nx.draw_networkx_nodes(G,pos=nodes_position,
                       node_size=20,
                      node_color='black',
                       alpha=0.3,
                      ax=ax3)

# Unidirectional edges
nx.draw_networkx_edges(G,pos=nodes_position,
                       edgelist = unidir,
                       node_size=20,
                       width=unidir_weight,
                       edge_color='black',
                       alpha=0.3,
                      ax=ax3)

# bidirectional edges
nx.draw_networkx_edges(G,pos=nodes_position,
                       node_size=20,
                       edgelist = bidir,
                       width=bidir_weight,
                       edge_color='black',
                       connectionstyle="arc3,rad=0.1",
                       alpha=0.2,
                      ax=ax3)



nx.draw_networkx_nodes(H3,pos=nodes_position_dual,
                       node_size=20,
                       node_color='red',
                      alpha= 1,
                      ax=ax3)

nx.draw_networkx_edges(H3,pos=nodes_position_dual,
                       node_size=20,
                       edgelist = widths3.keys(),
                       width=list(widths3.values()),
                       edge_color='red', 
                       connectionstyle="arc3,rad=0.2",
                      alpha = 1,
                      ax=ax3)
plt.tight_layout()